# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest
from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
import seaborn as sns
import pandas as pd
from scipy import stats

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
salaries.columns

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

In [6]:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
Annual = salaries['Annual Salary'].isnull().values.sum()
Hourly = salaries['Hourly Rate'].isnull().values.sum()
print(f"There are {Hourly} missing data in the hourly rates and {Annual} in the Annual Salary")


There are 25161 missing data in the hourly rates and 8022 in the Annual Salary


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
department_counts = salaries['Department'].value_counts()
print(department_counts)


Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
_, p_value = ztest(hourly_workers, value = 30, alternative = "two-sided")

if p_value < 0.05:
    print("We reject the null hypothesis. The hourly wage is significantly different from $30/hr.")
else:
    print("We fail to reject the null hypothesis. There is no significant difference from $30/hr.")


We reject the null hypothesis. The hourly wage is significantly different from $30/hr.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [10]:
wage_police = salaries[salaries['Department'] == 'POLICE']['Annual Salary'].dropna()

_, p_value = ztest(wage_police, value = 86000, alternative="larger")
if p_value < 0.05:
    print("We reject the null hypothesis. The salaries this year are higher than last year's mean of $86000/year")
else:
    print("We fail to reject the null hypothesis.The Salaries this year are not higher than last year's mean of $86000/year.")

We reject the null hypothesis. The salaries this year are higher than last year's mean of $86000/year


Using the `crosstab` function, find the department that has the most hourly workers. 

In [11]:
cross_tab = pd.crosstab(salaries["Department"], salaries["Salary or Hourly"] == "Hourly")
department_with_most_hourly_workers = cross_tab[True].idxmax()
print("Department with the most hourly workers:", department_with_most_hourly_workers)

Department with the most hourly workers: STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [18]:
wage_streets = salaries[salaries['Department'] == 'STREETS & SAN']['Annual Salary'].dropna()
t_stat, p_value = stats.ttest_1samp(wage_streets,35,alternative='less' )

if p_value < 0.05:
    print("We fail to reject the null hypothesis. Hourly wage is not significally different than $35/hr")
else:
    print("We reject the null hypothesis. Hourly wage is significally different than $35/hr")


We reject the null hypothesis. Hourly wage is significally different than $35/hr


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [23]:
data_per_year = salaries[salaries["Salary or Hourly"] == "Hourly"]["Hourly Rate"]

mean = np.mean(data_per_year)
std = np.std(data_per_year, ddof=1)
n = len(data_per_year)


standard_error = std / np.sqrt(n)
confidence_level = 0.95


confidence_interval = stats.t.interval(confidence_level, df= n - 1, loc = mean, scale=standard_error)

print("The Confidence Interval with a 95 % certainty, are between these two values: ", confidence_interval)


The Confidence Interval with a 95 % certainty, are between these two values:  (32.52345834488425, 33.05365708767623)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [21]:
data_police = salaries[salaries["Department"] == "POLICE"]["Annual Salary"].dropna()

mean = np.mean(data_police)
std = np.std(data_police, ddof=1)
size = len(data_police)


standard_error = std / np.sqrt(size)
confidence_level = 0.95


confidence_interval = stats.t.interval(confidence_level, df= size - 1, loc = mean, scale=standard_error)

print("The Confidence Interval with a 95 % certainty, are between these two values: ", confidence_interval)

The Confidence Interval with a 95 % certanty, are between these two values:  (86177.05631531784, 86795.77269094894)


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [29]:
from statsmodels.stats.proportion import proportions_ztest
counts = salaries["Salary or Hourly"][salaries["Salary or Hourly"]=="Hourly"].value_counts()[0]
number_observations = len(salaries["Salary or Hourly"])
_, p_value = proportions_ztest(counts, number_observations, value = 0.25)
p_value

h0 = "The Number of hourly workers in the City of Chicago is not significantly different from 25%. We keep H0."
h1 = "The Number of hourly workers in the City of Chicago is significantly different from 25%. We reject H0."

if p_value < 0.95: # It is very unlikely (p value is very low) that it happens, given  H0 is true 
    print(h1)
else:
    print(h0)

The Number of hourly workers in the City of Chicago is significantly different from 25%. We reject H0.


C:\Users\amirg\AppData\Local\Temp\ipykernel_5252\1753855777.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  counts = salaries["Salary or Hourly"][salaries["Salary or Hourly"]=="Hourly"].value_counts()[0]
